In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
import os

# Define o caminho do diretório da raiz do projeto
project_root = os.path.abspath(os.getcwd())
warehouse_path = os.path.join(project_root, 'data_aula')

# Inicializando a sessão do Spark com configurações do Hive
spark = SparkSession.builder \
    .appName("Iceberg Example") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hadoop") \
    .config("spark.sql.catalog.spark_catalog.warehouse",warehouse_path) \
    .config("datanucleus.schema.autoCreateTables", "true") \
    .getOrCreate()
    
# Criando a tabela de clientes
spark.sql("""
    CREATE TABLE spark_catalog.default.clientes (
        cd_cliente BIGINT,
        nome STRING,
        idade INT,
        cidade STRING,
        email STRING,
        data_nascimento DATE
    ) USING iceberg
""")

# Criando a tabela de carros com a chave estrangeira cd_cliente
spark.sql("""
    CREATE TABLE spark_catalog.default.carros (
        id BIGINT,
        cd_cliente BIGINT,
        marca STRING,
        modelo STRING,
        ano INT,
        preco DOUBLE,
        quilometragem BIGINT,
        cor STRING,
        tipo_combustivel STRING,
        cambio STRING
    ) USING iceberg
""")


# Inserindo dados de clientes
dados_clientes = [
    (1, "Ana Silva", 35, "São Paulo", "ana.silva@gmail.com", "1989-02-10"),
    (2, "Bruno Souza", 40, "Rio de Janeiro", "bruno.souza@gmail.com", "1984-06-21"),
    (3, "Carlos Pereira", 29, "Belo Horizonte", "carlos.pereira@yahoo.com", "1995-03-14"),
    (4, "Daniela Costa", 25, "Curitiba", "daniela.costa@outlook.com", "1999-09-07"),
    (5, "Eduardo Oliveira", 45, "Porto Alegre", "eduardo.oliveira@gmail.com", "1979-12-30"),
    (6, "Fernanda Lima", 32, "Fortaleza", "fernanda.lima@gmail.com", "1992-08-25"),
    (7, "Gabriel Mendes", 38, "Florianópolis", "gabriel.mendes@gmail.com", "1986-10-15"),
    (8, "Helena Ribeiro", 28, "Brasília", "helena.ribeiro@gmail.com", "1996-04-03"),
    (9, "Isabela Martins", 30, "Salvador", "isabela.martins@gmail.com", "1994-01-18"),
    (10, "João Santos", 27, "Recife", "joao.santos@gmail.com", "1997-11-22"),
    (11, "Karina Rocha", 36, "Manaus", "karina.rocha@gmail.com", "1988-05-09"),
    (12, "Leonardo Ferreira", 41, "São Paulo", "leonardo.ferreira@gmail.com", "1983-07-04"),
    (13, "Mariana Souza", 33, "Belo Horizonte", "mariana.souza@gmail.com", "1991-12-15"),
    (14, "Nathalia Teixeira", 39, "Curitiba", "nathalia.teixeira@gmail.com", "1985-09-12"),
    (15, "Otávio Moreira", 46, "Porto Alegre", "otavio.moreira@gmail.com", "1978-02-28"),
    (16, "Patrícia Cardoso", 31, "Fortaleza", "patricia.cardoso@gmail.com", "1993-06-19"),
    (17, "Ricardo Barbosa", 34, "Florianópolis", "ricardo.barbosa@gmail.com", "1990-10-23"),
    (18, "Sofia Gomes", 26, "Brasília", "sofia.gomes@gmail.com", "1998-04-13"),
    (19, "Thiago Alves", 42, "Salvador", "thiago.alves@gmail.com", "1982-11-01"),
    (20, "Vanessa Duarte", 29, "Recife", "vanessa.duarte@gmail.com", "1995-07-29")
]

# Criando o DataFrame para clientes
df_clientes = spark.createDataFrame(dados_clientes, ["cd_cliente", "nome", "idade", "cidade", "email", "data_nascimento"])

# Convertendo a coluna 'data_nascimento' para o tipo 'date'
df_clientes = df_clientes.withColumn("data_nascimento", to_date(df_clientes["data_nascimento"], "yyyy-MM-dd"))

# Gravando os dados na tabela Iceberg 'clientes'
df_clientes.writeTo("spark_catalog.default.clientes").append()

# Inserindo dados de carros, vinculando com clientes
dados_carros = [
    (1, 1, "Toyota", "Corolla", 2020, 75000.00, 15000, "Prata", "Gasolina", "Automático"),
    (2, 1, "Honda", "Civic", 2019, 72000.00, 20000, "Preto", "Flex", "Manual"),
    (3, 2, "Ford", "Fiesta", 2018, 55000.00, 30000, "Branco", "Gasolina", "Automático"),
    (4, 3, "Chevrolet", "Onix", 2021, 68000.00, 5000, "Vermelho", "Flex", "Automático"),
    (5, 4, "Volkswagen", "Golf", 2017, 90000.00, 40000, "Azul", "Gasolina", "Manual"),
    (6, 5, "Renault", "Sandero", 2022, 62000.00, 1000, "Cinza", "Flex", "Automático"),
    (7, 6, "Hyundai", "HB20", 2019, 69000.00, 12000, "Prata", "Gasolina", "Automático"),
    (8, 7, "Nissan", "Sentra", 2020, 83000.00, 18000, "Preto", "Gasolina", "Manual"),
    (9, 8, "Fiat", "Argo", 2021, 70000.00, 8000, "Vermelho", "Flex", "Automático"),
    (10, 9, "BMW", "320i", 2020, 150000.00, 22000, "Branco", "Gasolina", "Automático"),
    (11, 10, "Audi", "A3", 2021, 120000.00, 9000, "Cinza", "Gasolina", "Automático"),
    (12, 11, "Mercedes", "C180", 2020, 180000.00, 15000, "Preto", "Gasolina", "Automático"),
    (13, 12, "Chevrolet", "Cruze", 2019, 95000.00, 17000, "Branco", "Flex", "Automático"),
    (14, 13, "Ford", "Ka", 2018, 40000.00, 35000, "Vermelho", "Gasolina", "Manual"),
    (15, 14, "Hyundai", "Tucson", 2020, 100000.00, 18000, "Preto", "Gasolina", "Automático"),
    (16, 15, "Toyota", "Hilux", 2019, 190000.00, 25000, "Prata", "Diesel", "Automático"),
    (17, 16, "Jeep", "Compass", 2021, 150000.00, 10000, "Branco", "Diesel", "Automático"),
    (18, 17, "Fiat", "Uno", 2017, 35000.00, 45000, "Cinza", "Flex", "Manual"),
    (19, 18, "Honda", "Fit", 2018, 80000.00, 23000, "Preto", "Gasolina", "Automático"),
    (20, 19, "Volkswagen", "Polo", 2021, 85000.00, 12000, "Vermelho", "Flex", "Automático"),
    (21, 20, "Renault", "Duster", 2022, 90000.00, 5000, "Azul", "Gasolina", "Automático"),
    (22, 1, "Fiat", "Strada", 2020, 65000.00, 20000, "Branco", "Diesel", "Manual"),
    (23, 2, "Chevrolet", "S10", 2021, 180000.00, 8000, "Prata", "Diesel", "Automático"),
    (24, 3, "Volkswagen", "Jetta", 2020, 125000.00, 13000, "Preto", "Gasolina", "Automático"),
    (25, 4, "Honda", "HR-V", 2019, 90000.00, 24000, "Cinza", "Gasolina", "Automático"),
    (26, 5, "Hyundai", "Creta", 2021, 120000.00, 7000, "Branco", "Flex", "Automático"),
    (27, 6, "Toyota", "Yaris", 2020, 70000.00, 15000, "Prata", "Flex", "Automático"),
    (28, 7, "Ford", "Ranger", 2019, 150000.00, 30000, "Vermelho", "Diesel", "Automático"),
    (29, 8, "Chevrolet", "Spin", 2021, 85000.00, 10000, "Preto", "Flex", "Manual"),
    (30, 9, "BMW", "X1", 2020, 180000.00, 21000, "Azul", "Gasolina", "Automático"),
    (31, 10, "Audi", "Q3", 2021, 210000.00, 12000, "Cinza", "Gasolina", "Automático"),
    (32, 11, "Mercedes", "GLA", 2020, 230000.00, 8000, "Preto", "Gasolina", "Automático"),
    (33, 12, "Chevrolet", "Tracker", 2021, 95000.00, 5000, "Branco", "Flex", "Automático"),
    (34, 13, "Volkswagen", "T-Cross", 2020, 115000.00, 11000, "Prata", "Flex", "Automático"),
    (35, 14, "Hyundai", "Santa Fe", 2019, 150000.00, 17000, "Preto", "Gasolina", "Automático"),
    (36, 15, "Jeep", "Renegade", 2021, 120000.00, 6000, "Vermelho", "Diesel", "Automático"),
    (37, 16, "Renault", "Captur", 2020, 90000.00, 13000, "Branco", "Flex", "Automático"),
    (38, 17, "Fiat", "Mobi", 2018, 40000.00, 20000, "Cinza", "Flex", "Manual"),
    (39, 18, "Honda", "City", 2020, 90000.00, 22000, "Prata", "Gasolina", "Automático"),
    (40, 19, "Ford", "EcoSport", 2021, 85000.00, 9000, "Preto", "Flex", "Automático"),
    (41, 20, "Volkswagen", "Virtus", 2021, 95000.00, 8000, "Azul", "Flex", "Automático"),
    (42, 1, "Fiat", "Toro", 2020, 130000.00, 12000, "Vermelho", "Diesel", "Automático"),
    (43, 2, "Chevrolet", "Onix Plus", 2021, 75000.00, 3000, "Prata", "Flex", "Automático"),
    (44, 3, "Volkswagen", "Amarok", 2020, 170000.00, 14000, "Preto", "Diesel", "Automático"),
    (45, 4, "Honda", "Civic", 2019, 80000.00, 16000, "Branco", "Gasolina", "Automático"),
    (46, 5, "Hyundai", "i30", 2021, 85000.00, 4000, "Preto", "Gasolina", "Automático"),
    (47, 6, "Toyota", "Etios", 2019, 60000.00, 18000, "Prata", "Flex", "Manual"),
    (48, 7, "Ford", "Mustang", 2020, 300000.00, 7000, "Vermelho", "Gasolina", "Automático"),
    (49, 8, "Chevrolet", "Celta", 2017, 30000.00, 60000, "Branco", "Flex", "Manual"),
    (50, 9, "BMW", "X5", 2021, 350000.00, 5000, "Preto", "Gasolina", "Automático")
]

# Criando o DataFrame para carros
df_carros = spark.createDataFrame(dados_carros, ["id", "cd_cliente", "marca", "modelo", "ano", "preco", "quilometragem", "cor", "tipo_combustivel", "cambio"])

# Gravando os dados na tabela Iceberg 'carros'
df_carros.writeTo("spark_catalog.default.carros").append()

# Fazendo uma junção entre clientes e carros
spark.sql("""
    SELECT c.nome, c.cidade, a.marca, a.modelo, a.preco
    FROM spark_catalog.default.carros a
    JOIN spark_catalog.default.clientes c ON a.cd_cliente = c.cd_cliente
""").show()

24/10/04 13:20:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------------+--------------+----------+-------+--------+
|             nome|        cidade|     marca| modelo|   preco|
+-----------------+--------------+----------+-------+--------+
|        Ana Silva|     São Paulo|    Toyota|Corolla| 75000.0|
|        Ana Silva|     São Paulo|     Honda|  Civic| 72000.0|
|      Bruno Souza|Rio de Janeiro|      Ford| Fiesta| 55000.0|
|   Carlos Pereira|Belo Horizonte| Chevrolet|   Onix| 68000.0|
|    Daniela Costa|      Curitiba|Volkswagen|   Golf| 90000.0|
| Eduardo Oliveira|  Porto Alegre|   Renault|Sandero| 62000.0|
|    Fernanda Lima|     Fortaleza|   Hyundai|   HB20| 69000.0|
|   Gabriel Mendes| Florianópolis|    Nissan| Sentra| 83000.0|
|   Helena Ribeiro|      Brasília|      Fiat|   Argo| 70000.0|
|  Isabela Martins|      Salvador|       BMW|   320i|150000.0|
|      João Santos|        Recife|      Audi|     A3|120000.0|
|     Karina Rocha|        Manaus|  Mercedes|   C180|180000.0|
|Leonardo Ferreira|     São Paulo| Chevrolet|  Cruze| 9

#INCLUSAO DE CARRO

In [26]:
# Novos dados de carros para incluir
novos_dados_carros = [
    (51, 1, "Toyota", "Prius", 2022, 90000.00, 5000, "Branco", "Híbrido", "Automático"),
    (52, 2, "Tesla", "Model 3", 2023, 250000.00, 2000, "Preto", "Elétrico", "Automático")
]

# Criando DataFrame dos novos carros
df_novos_carros = spark.createDataFrame(novos_dados_carros, ["id", "cd_cliente", "marca", "modelo", "ano", "preco", "quilometragem", "cor", "tipo_combustivel", "cambio"])

# Incluindo os novos registros na tabela 'carros'
df_novos_carros.writeTo("spark_catalog.default.carros").append()

spark.sql("""
    SELECT * FROM spark_catalog.default.carros a
""").show()


+---+----------+----------+-------+----+--------+-------------+--------+----------------+----------+
| id|cd_cliente|     marca| modelo| ano|   preco|quilometragem|     cor|tipo_combustivel|    cambio|
+---+----------+----------+-------+----+--------+-------------+--------+----------------+----------+
|  1|         1|    Toyota|Corolla|2020| 75000.0|        15000|   Prata|        Gasolina|Automático|
|  2|         1|     Honda|  Civic|2019| 72000.0|        20000|   Preto|            Flex|    Manual|
|  3|         2|      Ford| Fiesta|2018| 55000.0|        30000|  Branco|        Gasolina|Automático|
|  4|         3| Chevrolet|   Onix|2021| 68000.0|         5000|Vermelho|            Flex|Automático|
|  5|         4|Volkswagen|   Golf|2017| 90000.0|        40000|    Azul|        Gasolina|    Manual|
|  6|         5|   Renault|Sandero|2022| 62000.0|         1000|   Cinza|            Flex|Automático|
|  7|         6|   Hyundai|   HB20|2019| 69000.0|        12000|   Prata|        Gasolina|Au

#ALTERAR TABELA DE CARROS

No Iceberg, nao existe um metodo de UPDATE como nos bancos tradicionais, precisando entao fazer o comando merge.

Outro exemplo de alteracao, seria sobrescrever os dados desejados da maneira abaixo:

# Lendo os dados da tabela 'carros'
df_carros = spark.table("spark_catalog.default.carros")

# Atualizando o preço do carro com 'id' 51
df_carros_atualizado = df_carros.withColumn(
    "preco", 
    when(df_carros.id == 51, 95000.00).otherwise(df_carros.preco)
)

# Sobrescrevendo a tabela com os dados atualizados
df_carros_atualizado.writeTo("spark_catalog.default.carros").overwrite()




In [27]:
spark.sql("""
    SELECT * FROM spark_catalog.default.carros a where a.id = 51
""").show()

spark.sql("""
    MERGE INTO spark_catalog.default.carros AS target
    USING (SELECT 51 AS id, 95000.00 AS novo_preco) AS updates
    ON target.id = updates.id
    WHEN MATCHED THEN UPDATE SET target.preco = updates.novo_preco
""")

spark.sql("""
    SELECT * FROM spark_catalog.default.carros a where a.id = 51
""").show()


+---+----------+------+------+----+-------+-------------+------+----------------+----------+
| id|cd_cliente| marca|modelo| ano|  preco|quilometragem|   cor|tipo_combustivel|    cambio|
+---+----------+------+------+----+-------+-------------+------+----------------+----------+
| 51|         1|Toyota| Prius|2022|90000.0|         5000|Branco|         Híbrido|Automático|
+---+----------+------+------+----+-------+-------------+------+----------------+----------+



UnsupportedOperationException: MERGE INTO TABLE is not supported temporarily.

#DELECAO DE CARROS

In [28]:
# Deletando o carro com 'id' 52
spark.sql("""
    SELECT * FROM spark_catalog.default.carros a
""").show()

spark.sql("""
    DELETE FROM spark_catalog.default.carros
    WHERE id = 52
""")

spark.sql("""
    SELECT * FROM spark_catalog.default.carros a
""").show()

DataFrame[]

#INCLUSAO DE CLIENTE

In [29]:
# Dados de novos clientes
novos_dados_clientes = [
    (21, "Pedro Nunes", 37, "São Paulo", "pedro.nunes@gmail.com", "1987-07-19"),
    (22, "Maria Clara", 28, "Rio de Janeiro", "maria.clara@gmail.com", "1996-11-22")
]

# Criando DataFrame dos novos clientes
df_novos_clientes = spark.createDataFrame(novos_dados_clientes, ["cd_cliente", "nome", "idade", "cidade", "email", "data_nascimento"])

# Convertendo a coluna 'data_nascimento' para o tipo 'date'
df_novos_clientes = df_novos_clientes.withColumn("data_nascimento", to_date(df_novos_clientes["data_nascimento"], "yyyy-MM-dd"))

# Incluindo os novos registros na tabela 'clientes'
df_novos_clientes.writeTo("spark_catalog.default.clientes").append()

spark.sql("""
    SELECT * FROM spark_catalog.default.clientes
""").show()


#ALTERACAO DE CLIENTE

No Iceberg, nao existe um metodo de UPDATE como nos bancos tradicionais, precisando entao fazer o comando merge.

Outro exemplo de alteracao, seria sobrescrever os dados desejados da maneira abaixo:

from pyspark.sql.functions import when

# Lendo os dados da tabela 'clientes'
df_clientes = spark.table("spark_catalog.default.clientes")

# Atualizando o email do cliente com 'cd_cliente' 21
df_clientes_atualizado = df_clientes.withColumn(
    "email", 
    when(df_clientes.cd_cliente == 21, "pedro.nunes@novodominio.com").otherwise(df_clientes.email)
)

# Sobrescrevendo a tabela com os dados atualizados
df_clientes_atualizado.writeTo("spark_catalog.default.clientes").overwrite()


In [30]:
# Atualizando o email do cliente com 'cd_cliente' 21
spark.sql("""
    MERGE INTO spark_catalog.default.clientes AS target
    USING (SELECT 21 AS cd_cliente, 'pedro.nunes@novodominio.com' AS novo_email) AS updates
    ON target.cd_cliente = updates.cd_cliente
    WHEN MATCHED THEN UPDATE SET target.email = updates.novo_email
""")


UnsupportedOperationException: MERGE INTO TABLE is not supported temporarily.

#DELECAO DE CLIENTE

In [31]:
spark.sql("""
    SELECT * FROM spark_catalog.default.clientes
""").show()


# Deletando o cliente com 'cd_cliente' 22
spark.sql("""
    DELETE FROM spark_catalog.default.clientes
    WHERE cd_cliente = 22
""")

spark.sql("""
    SELECT * FROM spark_catalog.default.clientes
""").show()


DataFrame[]